In [55]:
import numpy as np
import pandas as pd
import xlrd
import os
import tarfile
from six.moves import urllib
import seaborn as sns
import matplotlib.pyplot as plt

In [56]:
%matplotlib inline

In [57]:
pre = os.path.dirname(os.path.realpath('__file__'))
fname = 'CleanData.xlsx'
path = os.path.join(pre, fname)
data = pd.read_excel(path)

In [58]:
data.head()

,Powder,Sabot Depth,Muzzle,Sabot_OD,Actual_Velocity,Delta_Velocity,Desired_Velocity,Lp_Mass,Barrel_,Squeeze
0,163.0,4.250,0.489,0.4940,7.06,0.06,7,1.14580,16,0.0050
3,167.5,5.125,0.489,0.4935,6.96,-0.04,7,1.42045,16,0.0045
4,162.0,6.000,0.489,0.4935,6.89,-0.11,7,1.14705,16,0.0045
5,166.8,5.250,0.489,0.4930,7.02,0.02,7,1.35513,16,0.0040
6,158.0,6.750,0.489,0.4930,6.91,-0.09,7,1.04075,16,0.0040


In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157 entries, 0 to 390
Data columns (total 10 columns):
Powder              157 non-null float64
Sabot Depth         157 non-null float64
Muzzle              157 non-null float64
Sabot_OD            157 non-null float64
Actual_Velocity     157 non-null float64
Delta_Velocity      157 non-null float64
Desired_Velocity    157 non-null int64
Lp_Mass             157 non-null float64
Barrel_             157 non-null int64
Squeeze             157 non-null float64
dtypes: float64(8), int64(2)
memory usage: 13.5 KB


In [60]:
data.drop('Barrel_', axis=1,inplace=True)
data.drop('Delta_Velocity', axis=1,inplace=True)
data.drop('Desired_Velocity', axis=1,inplace=True)
data.drop('Actual_Velocity', axis=1,inplace=True)
#data.drop('Sabot_OD', axis=1,inplace=True)
data.drop('Sabot Depth', axis=1,inplace=True)
# data.drop('Squeeze', axis=1,inplace=True)
data.drop('Muzzle', axis=1,inplace=True)
data['Powder_'] = data['Powder']
data.drop('Powder', axis=1,inplace=True)

data.head()

,Sabot_OD,Lp_Mass,Squeeze,Powder_
0,0.4940,1.14580,0.0050,163.0
3,0.4935,1.42045,0.0045,167.5
4,0.4935,1.14705,0.0045,162.0
5,0.4930,1.35513,0.0040,166.8
6,0.4930,1.04075,0.0040,158.0


In [61]:
from sklearn.preprocessing import MinMaxScaler

In [62]:
scaler = MinMaxScaler()

In [63]:
scaler.fit(data.drop('Powder_', axis=1))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [64]:
scaled_features = scaler.transform(data.drop('Powder_', axis=1))

In [65]:
data_feat = pd.DataFrame(scaled_features, columns=data.columns[:-1])

In [66]:
data_feat.head()

,Sabot_OD,Lp_Mass,Squeeze
0,0.055907,0.092007,0.454545
1,0.050633,0.270263,0.409091
2,0.050633,0.092818,0.409091
3,0.045359,0.227868,0.363636
4,0.045359,0.023826,0.363636


In [67]:
from sklearn.model_selection import train_test_split

In [68]:
X = data_feat
y = data['Powder_']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)


In [69]:
from sklearn.neighbors import KNeighborsRegressor


In [70]:
knn = KNeighborsRegressor(n_neighbors=1)

In [71]:
knn.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=1, p=2,
          weights='uniform')

In [72]:
pred = knn.predict(X_test)

In [73]:
error_rate = []

In [74]:
# for i in range(1,10):
#     knn = KNeighborsRegressor(n_neighbors=i)
#     knn.fit(X_train,y_train)
#     pred_i = knn.predict(X_test)
#     error_rate.append(np.mean(pred_i != y_test))
    
    

In [75]:
# plt.figure(figsize=(10,6))
# plt.plot(range(1,10),error_rate,color='blue',linestyle='dashed',marker='o',markerfacecolor='red',markersize=10)
# plt.title='Error Rate vs K Value'
# plt.xlabel('K')
# plt.ylabel('Error Rate')


In [76]:
accuracy = knn.score(X_test, y_test)
print("Accuracy = {}%".format(accuracy*100))

Accuracy = 69.21906822617984%


In [77]:
import pickle

In [78]:
knn_regressionModel = open("KNNRegressionModel.pkl", "wb")

In [79]:
pickle.dump(knn, knn_regressionModel)

In [80]:
knn_regressionModel.close()

In [81]:
writer = pd.ExcelWriter('DataReadyforKNNModel.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
data.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()